# 0. Import needed Libraries

In [ ]:
# the 3 main libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from eofs.xarray import Eof as eof
# librairies to do nicer plots...
from matplotlib.patches import Rectangle    # only to draw a rectangle

# 1. Get data:

In [ ]:
#sstproduct = 'erssst'
sstproduct = 'oisst'

In [ ]:
if sstproduct == "erssst":
    
    data = xr.tutorial.open_dataset('ersstv5')
    data = data.sel(lat=slice(60., -60.))          # get rid of data out of 60S-60N
    data = data.sel(time=slice('1970','2021'))     # select years between 1970 and 2021
    mask = data.sst.isel(time=0)
    mask.data = np.where(np.isnan(mask.data), 0., 1.)
    
elif sstproduct == "oisst":
    
    data = xr.open_dataset('sst.mnmean.nc')
    data = data.sel(time=slice('1982','2022'))     # select years between 1982 and 2022
    data_mask = xr.open_dataset('lsmask.nc')
    mask = data_mask.mask                          # get the DataArray from the Dataset
    mask = mask.squeeze()                          # get rid of the time dimension in the variable mask
    
else:

    print("wrong name of SST product")

In [ ]:
data

In [ ]:
weights = np.cos( np.deg2rad(mask.lat) )
weights = mask*weights

# 2. linear trend:

Tip: Other options for 2D plots include [.plot.contour](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.contour.html), [.plot.contourf](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.contourf.html), [.plot.imshow](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.imshow.html)  
see also [colorbars list](https://matplotlib.org/tutorials/colors/colormaps.html)

### Long term mean trend
[linear regression](http://xarray.pydata.org/en/stable/computation.html?highlight=polyval#fitting-polynomials) with [polyfit](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.polyfit.html?highlight=polyfit) and [polyval](http://xarray.pydata.org/en/stable/generated/xarray.polyval.html?highlight=polyval) methonds

In [ ]:
linfit = data.sst.polyfit('time', 1)

### Detrend SST

In [ ]:
trend = xr.polyval(coord=data.time, coeffs=linfit.polyfit_coefficients)   # SST trend
sst_detrend = data.sst - trend.values + data.sst.mean(dim='time')         # detrended SST

### Detrended interannual anomaly

In [ ]:
sstbymth = sst_detrend.groupby("time.month")
mthclim = sstbymth.mean("time")       # detrended climatological months (seasonal cycle)
sstanom = sstbymth - mthclim          # detrended interannual annomaly
sstanom = sstanom.rename('sstanom')   # change variable name

# 3. SST Regression onto Nino3.4:

Interannual standard deviation, with [nino3.4](https://www.ncdc.noaa.gov/teleconnections/enso/indicators/sst/) box (5S-5N and 170-120W).

### Compute nino3.4 (5S-5N and 170-120W) index

In [ ]:
weights_nino34 = weights.sel(lon=slice(360-169.5,360-120.5), lat=slice(4.5,-4.5))
nino34_index = sstanom.weighted(weights_nino34).mean(dim=('lon','lat'))
nino34_index = nino34_index.rename('nino34')   # change variable name
nino34_index.plot()

### Monthly std of nino3.4 index

In [ ]:
nino34_index.groupby('time.month').std(dim='time').plot.step(where='mid')

### SST anomalies regressed onto Nino3.4 SST

In [ ]:
sstanom.coords["time"] = (("time"), nino34_index.data)  # redefine 'time' coordinates with nino34 time series
linfit = sstanom.polyfit('time', 1)                     # compute the regression with this new "time" 
sstanom.coords["time"] = (("time"), data.time.data)     # put back original time

sstreg = linfit.polyfit_coefficients.isel(degree=0)
sstcor = xr.corr(sstanom, nino34_index, dim='time')                      

#  create a 2 pannels figure
fig, axes = plt.subplots(1,2,figsize=(14, 4))
sstreg.where(mask == 1.).plot(ax=axes[0])                                # plot the regression coefficient
rect = Rectangle((360-170,-5),50,10,linewidth=1,edgecolor='w',fill=False)   # nino3.4 rectangle (170W-120W, 5S-5N)
axes[0].add_patch(rect)                                                     # overlay 
axes[0].set_title('Regression coefficient')
sstcor.where(mask == 1.).plot(ax=axes[1])                                # plot the correlation
rect = Rectangle((360-170,-5),50,10,linewidth=1,edgecolor='w',fill=False)   # nino3.4 rectangle (170W-120W, 5S-5N)
axes[1].add_patch(rect)                                                     # overlay 
axes[1].set_title('Correlation')
axes[1].set_xlabel('lon')
axes[1].set_ylabel('lat')

# 4. EOF of SST anomalies:

### Create an [Eof](https://ajdawson.github.io/eofs/latest/api/eofs.xarray.html#eofs.xarray.Eof) object

In [ ]:
solver = eof(sstanom, weights=weights)

### Explained variance by the different EOFs/PCs couple

In [ ]:
solver.varianceFraction(neigs=10).plot.stedata:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAkAAAAGxCAYAAACKvAkXAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8o6BhiAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAu1UlEQVR4nO3de1RV5b7/8c8CuSkXLwRqEqJmipoXSMVL6s5LdtmaDaUs7WI1rEwBszLL8srJTt4qNHalx/ausOy+NaVMzTRLAq2tRytTyGAQluAt0MX8/eFx/VqBCosFC3jerzHWGKxnzvXM73S1z/qc53nmnDbLsiwBAAAYxMvTBQAAANQ0AhAAADAOAQgAABiHAAQAAIxDAAIAAMYhAAEAAOMQgAAAgHEIQAAAwDgNPF1AbVRaWqpffvlFQUFBstlsni4HAABUgGVZOnbsmFq2bCkvrwuP8RCAyvHLL78oIiLC02UAAAAX5OTkqFWrVhfchwBUjqCgIEln/wGDg4M9XA0AAKiIoqIiRUREOH7HL4QAVI5z017BwcEEIAAA6piKLF9hETQAADAOAQgAABiHAAQAAIxDAAIAAMYhAAEAAOMQgAAAgHEIQAAAwDgEIAAAYBwCEAAAMA4BCAAAGIcABAAAjEMAAgAAxuFhqDXIsiydOm2vkWMF+HhX6GFwAACYiABUg06dtit65voaOVZsZBO9NTGOEAQAQDmYAqundh76vcZGmwAAqGsYAapBAT7e2jN7WLUe42SJXbFzP6nWYwAAUNcRgGqQzWZTQ1/+yQEA8DSmwAAAgHEIQAAAwDgEIAAAYBwCEAAAMA4BCAAAGIcABAAAjEMAAgAAxiEAAQAA4xCAAACAcQhAAADAOAQgAABgHAIQAAAwDgEIAAAYhwAEAACMQwACAADGIQABAADjEIAAAIBxCEAAAMA4BCAAAGAcAhAAADAOAQgAABiHAAQAAIxDAAIAAMYhAAEAAOMQgAAAgHEIQAAAwDgEIAAAYBwCEAAAMA4BCAAAGIcABAAAjEMAAgAAxiEAAQAA4xCAAACAcQhAAADAOAQgAABgHAIQAAAwDgEIAAAYhwAEAACMQwACAADGIQABAADjEIAAAIBxCEAAAMA4BCAAAGAcAhAAADAOAQgAABiHAAQAAIxTKwJQSkqKoqKi5O/vr5iYGH3++efn3fedd97RkCFDdMkllyg4OFhxcXFav359mf3WrFmj6Oho+fn5KTo6Wu+++251ngIAAKhDPB6A0tLSlJCQoBkzZigzM1P9+/fX8OHDlZ2dXe7+W7Zs0ZAhQ7R27VplZGRo0KBBuvHGG5WZmenYZ/v27YqPj9e4ceO0a9cujRs3TmPGjNGOHTtq6rQAAEAtZrMsy/JkAb169VKPHj20bNkyR1vHjh01cuRIJScnV6iPTp06KT4+XjNnzpQkxcfHq6ioSOvWrXPsc+2116pJkyZ64403LtpfUVGRQkJCVFhYqODg4EqekWedLDmj6JlnR8T2zB6mhr4NPFwRAAA1ozK/3x4dASopKVFGRoaGDh3q1D506FBt27atQn2Ulpbq2LFjatq0qaNt+/btZfocNmxYhfsEAAD1m0eHBwoKCmS32xUeHu7UHh4erry8vAr18dxzz+nEiRMaM2aMoy0vL69SfRYXF6u4uNjxvqioqKKnAAAA6iCPrwGSJJvN5vTesqwybeV544039PTTTystLU1hYWEu95mcnKyQkBDHKyIiopJnAAAA6hKPBqDQ0FB5e3uXGZnJz88vM4LzV2lpaZowYYJWr16twYMHO21r3rx5pfqcPn26CgsLHa+cnBwXzgYAANQVHg1Avr6+iomJUXp6ulN7enq6+vTpc97PvfHGG7rzzjv1+uuv6/rrry+zPS4urkyfGzZsOG+ffn5+Cg4OdnoBAID6y+OXCCUlJWncuHGKjY1VXFycUlNTlZ2drYkTJ0o6Ozpz+PBhrVq1StLZ8DN+/HgtWbJEvXv3doz0BAQEKCQkRJI0ZcoUXX311XrmmWc0YsQIvf/++/rkk0+0detWz5wkAACoVTy+Big+Pl6LFy/W7Nmz1a1bN23ZskVr165VZGSkJCk3N9fpnkAvvfSSzpw5owcffFAtWrRwvKZMmeLYp0+fPnrzzTe1YsUKXXnllVq5cqXS0tLUq1evGj8/AABQ+3j8PkC1EfcBAgCg7qkz9wECAADwBAIQAAAwDgEIAAAYhwAEAACMQwACAADGIQABAADjEIAAAIBxCEAAAMA4BCAAAGAcAhAAADAOAQgAABiHAAQAAIxDAAIAAMYhAAEAAOMQgAAAgHEIQAAAwDgEIAAAYBwCEAAAMA4BCAAAGIcABAAAjEMAAgAAxiEAAQAA4xCAAACAcQhAAADAOAQgAABgHAIQAAAwDgEIAAAYhwAEAACMQwACAADGIQABAADjEIAAAIBxCEAAAMA4BCAAAGAcAhAAADAOAQgAABiHAAQAAIxDAAIAAMYhAAEAAOMQgAAAgHEIQAAAwDgEIAAAYBwCEAAAMA4BCAAAGIcABAAAjEMAAgAAxiEAAQAA47gtAB09etRdXQEAAFQrlwLQM888o7S0NMf7MWPGqFmzZrr00ku1a9cutxUHAABQHVwKQC+99JIiIiIkSenp6UpPT9e6des0fPhwTZs2za0FAgAAuFsDVz6Um5vrCEAfffSRxowZo6FDh6p169bq1auXWwsEAABwN5dGgJo0aaKcnBxJ0scff6zBgwdLkizLkt1ud191AAAA1cClEaBRo0Zp7Nixuvzyy3XkyBENHz5ckpSVlaV27dq5tUAAAAB3cykALVq0SK1bt1ZOTo4WLFigwMBASWenxh544AG3FggAAOBuLgUgHx8fPfzww2XaExISqloPAABAtXMpAEnS/v37tWnTJuXn56u0tNRp28yZM6tcGAAAQHVxKQD94x//0P3336/Q0FA1b95cNpvNsc1msxGAAABAreZSAJo7d67mzZunRx991N31AAAAVDuXLoP//fffNXr0aHfXAgAAUCNcCkCjR4/Whg0b3F0LAABAjXBpCqxdu3Z68skn9eWXX6pLly7y8fFx2j558mS3FAcAAFAdXApAqampCgwM1ObNm7V582anbTabjQAEAABqNZcC0E8//eTuOgAAAGqMS2uA/syyLFmW5Y5aAAAAaoTLAWjVqlXq0qWLAgICFBAQoCuvvFKvvfaaO2sDAACoFi5NgS1cuFBPPvmkJk2apL59+8qyLH3xxReaOHGiCgoKlJiY6O46AQAA3MalAPT8889r2bJlGj9+vKNtxIgR6tSpk55++mkCEAAAqNVcmgLLzc1Vnz59yrT36dNHubm5VS4KAACgOrkUgNq1a6fVq1eXaU9LS9Pll19e6f5SUlIUFRUlf39/xcTE6PPPPz/vvrm5uRo7dqyuuOIKeXl5lfsE+pUrV8pms5V5/fHHH5WuDQAA1D8uTYHNmjVL8fHx2rJli/r27SubzaatW7fq008/LTcYXUhaWpoSEhKUkpKivn376qWXXtLw4cO1Z88eXXbZZWX2Ly4u1iWXXKIZM2Zo0aJF5+03ODhY+/btc2rz9/evVG0AAKB+cmkE6Oabb9aOHTsUGhqq9957T++8845CQ0P11Vdf6aabbqpUXwsXLtSECRN0zz33qGPHjlq8eLEiIiK0bNmycvdv3bq1lixZovHjxyskJOS8/dpsNjVv3tzpBQAAILk4AiRJMTEx+uc//1mlg5eUlCgjI0OPPfaYU/vQoUO1bdu2KvV9/PhxRUZGym63q1u3bpozZ466d+9epT4BAED9UOEAVFRUpODgYMffF3Juv4spKCiQ3W5XeHi4U3t4eLjy8vIqWloZHTp00MqVK9WlSxcVFRVpyZIl6tu3r3bt2lXuGqXi4mIVFxc73l/s/AAAQN1W4QDUpEkT5ebmKiwsTI0bN5bNZiuzj2VZstlsstvtlSrir32d68dVvXv3Vu/evR3v+/btqx49euj555/X0qVLy+yfnJysWbNmuXw8AABQt1Q4AG3cuFFNmzaVJH322WduOXhoaKi8vb3LjPbk5+eXGRWqCi8vL1111VX6/vvvy90+ffp0JSUlOd4XFRUpIiLCbccHAAC1S4UD0IABAxx/R0VFKSIiotyRm5ycnAof3NfXVzExMUpPT3daPJ2enq4RI0ZUuJ+LsSxLWVlZ6tKlS7nb/fz85Ofn57bjAQCA2s2lRdBRUVGO6bA/++233xQVFVWpKbCkpCSNGzdOsbGxiouLU2pqqrKzszVx4kRJZ0dnDh8+rFWrVjk+k5WVJensQudff/1VWVlZ8vX1VXR0tKSzl+n37t1bl19+uYqKirR06VJlZWXpxRdfdOV0AQBAPeNSADrfGp3jx49X+l478fHxOnLkiGbPnq3c3Fx17txZa9euVWRkpKSzNz7Mzs52+syfr+bKyMjQ66+/rsjISB08eFCSdPToUd13333Ky8tTSEiIunfvri1btqhnz56VPFMAAFAf2SzLsiq687l1MkuWLNG9996rhg0bOrbZ7Xbt2LFD3t7e+uKLL9xfaQ0qKipSSEiICgsLK3xFW21xsuSMomeulyTtmT1MDX1dvtMBAAB1SmV+vyv165iZmSnp7AjQt99+K19fX8c2X19fde3aVQ8//LALJQMAANScSgWgc1d/3XXXXVqyZEmdGx0BAACQXHwUxuLFi3XmzJky7b/99hs3EQQAALWeSwHolltu0ZtvvlmmffXq1brllluqXBQAAEB1cikA7dixQ4MGDSrTPnDgQO3YsaPKRQEAAFQnlwJQcXFxuVNgp0+f1qlTp6pcFAAAQHVyKQBdddVVSk1NLdO+fPlyxcTEVLkoAACA6uTSTWLmzZunwYMHa9euXbrmmmskSZ9++qm+/vprbdiwwa0FAgAAuJtLI0B9+/bV9u3bFRERodWrV+vDDz9Uu3bttHv3bvXv39/dNQIAALiVy7cJ7tatm/71r3+5sxYAAIAaUeXnJJw6dUqnT592auMGiQAAoDZzaQrs5MmTmjRpksLCwhQYGKgmTZo4vQAAAGozlwLQtGnTtHHjRqWkpMjPz08vv/yyZs2apZYtW2rVqlXurhEAAMCtXJoC+/DDD7Vq1SoNHDhQd999t/r376927dopMjJS//rXv3Tbbbe5u04AAAC3cWkE6LffflNUVJSks+t9fvvtN0lSv379tGXLFvdVBwAAUA1cCkBt2rTRwYMHJUnR0dFavXq1pLMjQ40bN3ZXbQAAANXCpQB01113adeuXZKk6dOnO9YCJSYmatq0aW4tEAAAwN1cWgOUmJjo+HvQoEH63//9X+3cuVNt27ZV165d3VYcAABAdaj0CNDp06c1aNAg7d+/39F22WWXadSoUYQfAABQJ1Q6APn4+Oi7776TzWarjnoAAACqnUtrgMaPH69XXnnF3bUAAADUCJfWAJWUlOjll19Wenq6YmNj1ahRI6ftCxcudEtxAAAA1cGlAPTdd9+pR48ekuS0FkgSU2MAAKDWq3AA2r17tzp37iwvLy999tln1VkTAABAtarwGqDu3buroKBA0tkbIR45cqTaigIAAKhOFQ5AjRs31k8//SRJOnjwoEpLS6utKAAAgOpU4Smwm2++WQMGDFCLFi1ks9kUGxsrb2/vcvc9cOCA2woEAABwtwoHoNTUVI0aNUo//PCDJk+erHvvvVdBQUHVWRsAAEC1qNRVYNdee60kKSMjQ1OmTLloAPr555/VsmVLeXm5dLshAACAauFSMlmxYkWFRn+io6MdT40HAACoLap1aMayrOrsHgAAwCXMTQEAAOMQgAAAgHEIQAAAwDjVGoB4LhgAAKiNWAQNAACMU6UA9MMPP2j9+vU6deqUpLKBZ8+ePYqMjKzKIQAAANzOpQB05MgRDR48WO3bt9d1112n3NxcSdI999yjqVOnOvaLiIg47+MyAAAAPMWlAJSYmKgGDRooOztbDRs2dLTHx8fr448/dltxAAAA1aFSj8I4Z8OGDVq/fr1atWrl1H755Zfr0KFDbikMAACgurg0AnTixAmnkZ9zCgoK5OfnV+WiAAAAqpNLAejqq6/WqlWrHO9tNptKS0v17LPPatCgQW4rDgAAoDq4NAX27LPPauDAgdq5c6dKSkr0yCOP6D//+Y9+++03ffHFF+6uEQAAwK1cGgGKjo7W7t271bNnTw0ZMkQnTpzQqFGjlJmZqbZt27q7RgAAALdyaQRIkpo3b65Zs2a5sxYAAIAa4dII0IoVK/TWW2+VaX/rrbf0P//zP1UuCgAAoDq5FID+67/+S6GhoWXaw8LCNH/+/CoXBQAAUJ1cCkCHDh1SVFRUmfbIyEhlZ2dXuSgAAIDq5FIACgsL0+7du8u079q1S82aNatyUQAAANXJpQB0yy23aPLkyfrss89kt9tlt9u1ceNGTZkyRbfccou7awQAAHArl64Cmzt3rg4dOqRrrrlGDRqc7aK0tFTjx49nDRAAAKj1XApAvr6+SktL05w5c7Rr1y4FBASoS5cuioyMdHd9AAAAbufyfYAkqX379mrfvr27agEAAKgRLgUgu92ulStX6tNPP1V+fr5KS0udtm/cuNEtxQEAAFQHlwLQlClTtHLlSl1//fXq3LmzbDabu+sCAACoNi4FoDfffFOrV6/Wdddd5+56AAAAqp1Ll8H7+vqqXbt27q4FAACgRrgUgKZOnaolS5bIsix31wMAAFDtXJoC27p1qz777DOtW7dOnTp1ko+Pj9P2d955xy3FAQAAVAeXAlDjxo110003ubsWAACAGuFSAFqxYoW76wAAAKgxLq0BAgAAqMtcvhP022+/rdWrVys7O1slJSVO27755psqFwYAAFBdXBoBWrp0qe666y6FhYUpMzNTPXv2VLNmzXTgwAENHz7c3TUCAAC4lUsBKCUlRampqXrhhRfk6+urRx55ROnp6Zo8ebIKCwvdXSMAAIBbuRSAsrOz1adPH0lSQECAjh07JkkaN26c3njjDfdVBwAAUA1cCkDNmzfXkSNHJEmRkZH68ssvJUk//fQTN0cEAAC1nksB6G9/+5s+/PBDSdKECROUmJioIUOGKD4+nvsDAQCAWs+lAJSamqoZM2ZIkiZOnKiVK1eqY8eOmjVrlpYtW1bp/lJSUhQVFSV/f3/FxMTo888/P+++ubm5Gjt2rK644gp5eXkpISGh3P3WrFmj6Oho+fn5KTo6Wu+++26l6wIAAPWTSwHIy8tLDRr8/yvox4wZo6VLl2ry5Mny9fWtVF9paWlKSEjQjBkzlJmZqf79+2v48OHKzs4ud//i4mJdcsklmjFjhrp27VruPtu3b1d8fLzGjRunXbt2ady4cRozZox27NhRqdoAAED9ZLMquGhn9+7d6ty5s7y8vLR79+4L7nvllVdWuIBevXqpR48eTiNHHTt21MiRI5WcnHzBzw4cOFDdunXT4sWLndrj4+NVVFSkdevWOdquvfZaNWnSpEKLtIuKihQSEqLCwkIFBwdX+Fxqg5MlZxQ9c70kac/sYWro6/KtngAAqFMq8/td4V/Hbt26KS8vT2FhYerWrZtsNlu5C55tNpvsdnuF+iwpKVFGRoYee+wxp/ahQ4dq27ZtFS2tjO3btysxMdGpbdiwYWWC0jnFxcUqLi52vC8qKnL52AAAoParcAD66aefdMkllzj+doeCggLZ7XaFh4c7tYeHhysvL8/lfvPy8irVZ3JysmbNmuXy8QAAQN1S4TVAkZGRstlsOn36tJ5++mnZ7XZFRkaW+6osm83m9N6yrDJt1dnn9OnTVVhY6Hjl5ORU6dgAAKB2q/QiaB8fH7ddURUaGipvb+8yIzP5+fllRnAqo3nz5pXq08/PT8HBwU4vAABQf7l0FdhNN92k9957r8oH9/X1VUxMjNLT053a09PTHXeadkVcXFyZPjds2FClPgEAQP3h0iVC7dq105w5c7Rt2zbFxMSoUaNGTtsnT55c4b6SkpI0btw4xcbGKi4uTqmpqcrOztbEiRMlnZ2eOnz4sFatWuX4TFZWliTp+PHj+vXXX5WVlSVfX19FR0dLkqZMmaKrr75azzzzjEaMGKH3339fn3zyibZu3erK6QIAgHqmwpfB/1lUVNT5O7TZdODAgUr1l5KSogULFig3N1edO3fWokWLdPXVV0uS7rzzTh08eFCbNm1yOsZfRUZG6uDBg473b7/9tp544gkdOHBAbdu21bx58zRq1KgK1cNl8AAA1D2V+f12KQDVdwQgAADqnsr8fru0BggAAKAuc3l44Oeff9YHH3yg7OxslZSUOG1buHBhlQsDAACoLi4FoE8//VR///vfFRUVpX379qlz5846ePCgLMtSjx493F0jAACAW7k0BTZ9+nRNnTpV3333nfz9/bVmzRrl5ORowIABGj16tLtrBAAAcCuXAtDevXt1xx13SJIaNGigU6dOKTAwULNnz9Yzzzzj1gIBAADczaUA1KhRI8fDQ1u2bKkff/zRsa2goMA9lQEAAFQTl9YA9e7dW1988YWio6N1/fXXa+rUqfr222/1zjvvqHfv3u6uEQAAwK1cCkALFy7U8ePHJUlPP/20jh8/rrS0NLVr106LFi1ya4EAAADu5lIAmjNnjm6//XZZlqWGDRsqJSXF3XUBAABUG5fWAB05ckTXX3+9WrVqpalTpzqezQUAAFAXuBSAPvjgA+Xl5empp55SRkaGYmJiFB0drfnz5zs9jwsAAKA2cvlRGI0bN9Z9992nTZs26dChQ7rrrrv02muvqV27du6sDwAAwO2q/Cyw06dPa+fOndqxY4cOHjyo8PBwd9QFAABQbVwOQJ999pnuvfdehYeH64477lBQUJA+/PBD5eTkuLM+AAAAt3PpKrBWrVrpyJEjGjZsmF566SXdeOON8vf3d3dtAAAA1cKlADRz5kyNHj1aTZo0cXc9AAAA1c6lAHTfffe5uw4AAIAaU+VF0AAAAHUNAQgAABiHAAQAAIxDAAIAAMYhAAEAAOMQgAAAgHEIQAAAwDgEIAAAYBwCEAAAMA4BCAAAGIcABAAAjEMAAgAAxiEAAQAA4xCAAACAcQhAAADAOAQgAABgHAIQAAAwDgEIAAAYp4GnC0D1OVli93QJVRbg4y2bzebpMgAA9QwBqB6LnfuJp0uostjIJnprYhwhCADgVkyB1TMBPt6KjWzi6TLcZueh33XqdN0fyQIA1C6MANUzNptNb02Mq/Oh4WSJvV6MYAEAaicCUD1ks9nU0JevFgCA82EKDAAAGIcABAAAjEMAAgAAxiEAAQAA4xCAAACAcQhAAADAOAQgAABgHAIQAAAwDgEIAAAYhwAEAACMQwACAADGIQABAADjEIAAAIBxCEAAAMA4BCAAAGAcAhAAADAOAQgAABiHAAQAAIxDAAIAAMYhAAEAAOMQgAAAgHEIQAAAwDgEIAAAYBwCEAAAMA4BCAAAGIcABAAAjEMAAgAAxqkVASglJUVRUVHy9/dXTEyMPv/88wvuv3nzZsXExMjf319t2rTR8uXLnbavXLlSNputzOuPP/6oztMAAAB1hMcDUFpamhISEjRjxgxlZmaqf//+Gj58uLKzs8vd/6efftJ1112n/v37KzMzU48//rgmT56sNWvWOO0XHBys3Nxcp5e/v39NnBIAAKjlGni6gIULF2rChAm65557JEmLFy/W+vXrtWzZMiUnJ5fZf/ny5brsssu0ePFiSVLHjh21c+dO/fd//7duvvlmx342m03NmzevkXMAAAB1i0dHgEpKSpSRkaGhQ4c6tQ8dOlTbtm0r9zPbt28vs/+wYcO0c+dOnT592tF2/PhxRUZGqlWrVrrhhhuUmZl53jqKi4tVVFTk9AIAAPWXRwNQQUGB7Ha7wsPDndrDw8OVl5dX7mfy8vLK3f/MmTMqKCiQJHXo0EErV67UBx98oDfeeEP+/v7q27evvv/++3L7TE5OVkhIiOMVERHhhrMDAAC1lcfXAElnp6v+zLKsMm0X2//P7b1799btt9+url27qn///lq9erXat2+v559/vtz+pk+frsLCQscrJyenKqcDAABqOY+uAQoNDZW3t3eZ0Z78/PwyozznNG/evNz9GzRooGbNmpX7GS8vL1111VXnHQHy8/OTn5+fC2cAAADqIo+OAPn6+iomJkbp6elO7enp6erTp0+5n4mLiyuz/4YNGxQbGysfH59yP2NZlrKystSiRQv3FA4AAOo0j0+BJSUl6eWXX9arr76qvXv3KjExUdnZ2Zo4caKks9NT48ePd+w/ceJEHTp0SElJSdq7d69effVVvfLKK3r44Ycd+8yaNUvr16/XgQMHlJWVpQkTJigrK8vRJwAAMJvHL4OPj4/XkSNHNHv2bOXm5qpz585au3atIiMjJUm5ublO9wSKiorS2rVrlZiYqBdffFEtW7bU0qVLnS6BP3r0qO677z7l5eUpJCRE3bt315YtW9SzZ88aPz9U3ckSu6dLqLIAH+8LrmsDANQsm3VuBTEcioqKFBISosLCQgUHB3u6HCOdLDmj6JnrPV2G28RGNtFbE+MIQQBQjSrz++3xKTCgPAE+3oqNbOLpMtxm56Hfdep03R/JAoD6wuNTYEB5bDab3poYV+dDw8kSu2LnfuLpMgAAf0EAQq1ls9nU0Jf/RAEA7scUGAAAMA4BCAAAGIcABAAAjEMAAgAAxiEAAQAA4xCAAACAcQhAAADAONxkBaghPNMMAGoPAhBQQ+rDHaF5phmA+oIpMKAa8UwzAKidGAECqhHPNAOA2okABFQznmkGALUPU2AAAMA4BCAAAGAcAhAAADAOCxMAVAr3MwJQHxCAAFRKfbgajPsZAWAKDMBFcT8jAPUNI0AALor7GQGobwhAACqkvt3PqD6sZZJYzwS4qv78XzMAqIT6MhLEeibANawBAmCM+raWSWI9E+AqRoAAGKO+rGWSWM8EVBUBCIBR6ttaJgCuYQoMAAAYh/83CADquPpwRRtXs6GmEYAAoI6rD2uBolsE/9/VbJ6upGoIcnUHAQgA6qBzV7TtPPS7p0txiz25Rer01HpPl1Fl3Jag7iAAAUAdVF+uaLMsafTy7dqTW+TpUtzi3G0JWGhf+/ENAUAdVV+uaPv35H51PshxW4K6p+7/LwcAUKfVlyB3DovS64b6818cAAC1QH0YCTJhLRP3AQIAoIrq22NWTHjECiNAAABUUX1ZlG7SWiYCEAAAblDf1jLVd0yBAQAA4xBVAQBAGTVxNZsnrzYjAAEAgDJqYi3QntnDPDZtyBQYAACQVP+uZrsQRoAAAICkmr+aLcDHu0aOUx4CEAAAcDDlajamwAAAgHEIQAAAwDgEIAAAYBwCEAAAMA4BCAAAGIcABAAAjEMAAgAAxiEAAQAA4xCAAACAcQhAAADAOAQgAABgHAIQAAAwDgEIAAAYp/4/7tUFlmVJkoqKijxcCQAAqKhzv9vnfscvhABUjmPHjkmSIiIiPFwJAACorGPHjikkJOSC+9isisQkw5SWluqXX35RUFCQbDabW/suKipSRESEcnJyFBwc7Na+UXl8H7UL30ftw3dSu/B9XJhlWTp27JhatmwpL68Lr/JhBKgcXl5eatWqVbUeIzg4mP94axG+j9qF76P24TupXfg+zu9iIz/nsAgaAAAYhwAEAACMQwCqYX5+fnrqqafk5+fn6VIgvo/ahu+j9uE7qV34PtyHRdAAAMA4jAABAADjEIAAAIBxCEAAAMA4BKAalJKSoqioKPn7+ysmJkaff/65p0syVnJysq666ioFBQUpLCxMI0eO1L59+zxdFv5PcnKybDabEhISPF2KsQ4fPqzbb79dzZo1U8OGDdWtWzdlZGR4uiwjnTlzRk888YSioqIUEBCgNm3aaPbs2SotLfV0aXUaAaiGpKWlKSEhQTNmzFBmZqb69++v4cOHKzs729OlGWnz5s168MEH9eWXXyo9PV1nzpzR0KFDdeLECU+XZryvv/5aqampuvLKKz1dirF+//139e3bVz4+Plq3bp327Nmj5557To0bN/Z0aUZ65plntHz5cr3wwgvau3evFixYoGeffVbPP/+8p0ur07gKrIb06tVLPXr00LJlyxxtHTt21MiRI5WcnOzByiBJv/76q8LCwrR582ZdffXVni7HWMePH1ePHj2UkpKiuXPnqlu3blq8eLGnyzLOY489pi+++IJR6lrihhtuUHh4uF555RVH280336yGDRvqtdde82BldRsjQDWgpKREGRkZGjp0qFP70KFDtW3bNg9VhT8rLCyUJDVt2tTDlZjtwQcf1PXXX6/Bgwd7uhSjffDBB4qNjdXo0aMVFham7t276x//+IenyzJWv3799Omnn2r//v2SpF27dmnr1q267rrrPFxZ3cazwGpAQUGB7Ha7wsPDndrDw8OVl5fnoapwjmVZSkpKUr9+/dS5c2dPl2OsN998U998842+/vprT5divAMHDmjZsmVKSkrS448/rq+++kqTJ0+Wn5+fxo8f7+nyjPPoo4+qsLBQHTp0kLe3t+x2u+bNm6dbb73V06XVaQSgGvTXJ8tbluX2p82j8iZNmqTdu3dr69atni7FWDk5OZoyZYo2bNggf39/T5djvNLSUsXGxmr+/PmSpO7du+s///mPli1bRgDygLS0NP3zn//U66+/rk6dOikrK0sJCQlq2bKl7rjjDk+XV2cRgGpAaGiovL29y4z25OfnlxkVQs166KGH9MEHH2jLli1q1aqVp8sxVkZGhvLz8xUTE+Nos9vt2rJli1544QUVFxfL29vbgxWapUWLFoqOjnZq69ixo9asWeOhisw2bdo0PfbYY7rlllskSV26dNGhQ4eUnJxMAKoC1gDVAF9fX8XExCg9Pd2pPT09XX369PFQVWazLEuTJk3SO++8o40bNyoqKsrTJRntmmuu0bfffqusrCzHKzY2VrfddpuysrIIPzWsb9++ZW4LsX//fkVGRnqoIrOdPHlSXl7OP9fe3t5cBl9FjADVkKSkJI0bN06xsbGKi4tTamqqsrOzNXHiRE+XZqQHH3xQr7/+ut5//30FBQU5RudCQkIUEBDg4erMExQUVGb9VaNGjdSsWTPWZXlAYmKi+vTpo/nz52vMmDH66quvlJqaqtTUVE+XZqQbb7xR8+bN02WXXaZOnTopMzNTCxcu1N133+3p0uo0LoOvQSkpKVqwYIFyc3PVuXNnLVq0iEuuPeR8a69WrFihO++8s2aLQbkGDhzIZfAe9NFHH2n69On6/vvvFRUVpaSkJN17772eLstIx44d05NPPql3331X+fn5atmypW699VbNnDlTvr6+ni6vziIAAQAA47AGCAAAGIcABAAAjEMAAgAAxiEAAQAA4xCAAACAcQhAAADAOAQgAABgHAIQAAAwDgEIgEssy9J9992npk2bymazKSsry9MlOQwcOFAJCQmeLqPKVq5cqcaNG3u6DKBe4llgAFzy8ccfa+XKldq0aZPatGmj0NBQT5cEABVGAALgkh9//FEtWrRQnz59PF0KKun06dPy8fHxdBmARzEFBhjKsiwtWLBAbdq0UUBAgLp27aq3337bsX3z5s3q2bOn/Pz81KJFCz322GM6c+aMJOnOO+/UQw89pOzsbNlsNrVu3fqixxs4cKAeeughJSQkqEmTJgoPD1dqaqpOnDihu+66S0FBQWrbtq3WrVvn9LkL1SFJJ06c0Pjx4xUYGKgWLVroueeeK3PskpISPfLII7r00kvVqFEj9erVS5s2barQv9O5aaj169erY8eOCgwM1LXXXqvc3Fync/vrlNvIkSOdHqzbunVrzZ0711FrZGSk3n//ff36668aMWKEAgMD1aVLF+3cubNMDe+9957at28vf39/DRkyRDk5OU7bP/zwQ8XExMjf319t2rTRrFmznP6NbDabli9frhEjRqhRo0aaO3duhc4dqNcsAEZ6/PHHrQ4dOlgff/yx9eOPP1orVqyw/Pz8rE2bNlk///yz1bBhQ+uBBx6w9u7da7377rtWaGio9dRTT1mWZVlHjx61Zs+ebbVq1crKzc218vPzL3q8AQMGWEFBQdacOXOs/fv3W3PmzLG8vLys4cOHW6mpqdb+/fut+++/32rWrJl14sQJy7Ksi9ZhWZZ1//33W61atbI2bNhg7d6927rhhhuswMBAa8qUKY59xo4da/Xp08fasmWL9cMPP1jPPvus5efnZ+3fv/+ida9YscLy8fGxBg8ebH399ddWRkaG1bFjR2vs2LFO5/bn41mWZY0YMcK64447HO8jIyOtpk2bWsuXL3eca1BQkHXttddaq1evtvbt22eNHDnS6tixo1VaWup07NjYWGvbtm3Wzp07rZ49e1p9+vRx9Pvxxx9bwcHB1sqVK60ff/zR2rBhg9W6dWvr6aefduwjyQoLC7NeeeUV68cff7QOHjx40fMG6jsCEGCg48ePW/7+/ta2bduc2idMmGDdeuut1uOPP25dccUVjh9iy7KsF1980QoMDLTsdrtlWZa1aNEiKzIyssLHHDBggNWvXz/H+zNnzliNGjWyxo0b52jLzc21JFnbt2+3LMu6aB3Hjh2zfH19rTfffNOx/ciRI1ZAQIAjkPzwww+WzWazDh8+7FTPNddcY02fPv2ida9YscKSZP3www9ONYSHhzudW0UC0O23317mXJ988klH2/bt2y1JVm5urtOxv/zyS8c+e/futSRZO3bssCzLsvr372/Nnz/f6divvfaa1aJFC8d7SVZCQsJFzxUwCWuAAAPt2bNHf/zxh4YMGeLUXlJSou7du+uPP/5QXFycbDabY1vfvn11/Phx/fzzz7rssstcOu6VV17p+Nvb21vNmjVTly5dHG3h4eGSpPz8fEnS3r17L1jH77//rpKSEsXFxTm2N23aVFdccYXj/TfffCPLstS+fXunWoqLi9WsWbMK1d2wYUO1bdvW8b5FixaOGivjz+d/7lzPd/7NmzeXJDVo0ECxsbGOfTp06KDGjRtr79696tmzpzIyMvT1119r3rx5jn3sdrv++OMPnTx5Ug0bNpQkpz4AsAgaMFJpaakk6d///rcuvfRSp21+fn6aPHmyU+iQzq4ZklSmvTL+uvDWZrM5tZ3r+1x9lmVdsI5zf19IaWmpvL29lZGRIW9vb6dtgYGBLtf952N7eXmVqeX06dMX7OfceV3o/P/aXl5baWmpZs2apVGjRpXZx9/f3/F3o0aNymwHTEYAAgwUHR0tPz8/ZWdna8CAAeVuX7NmjVMA2bZtm4KCgsoEpuqu80J1NGnSRD4+Pvryyy8do1K///679u/f7ziv7t27y263Kz8/X/3796+WOi+55BKnRdF2u13fffedBg0aVOW+z5w5o507d6pnz56SpH379uno0aPq0KGDJKlHjx7at2+f2rVrV+VjASYhAAEGCgoK0sMPP6zExESVlpaqX79+Kioq0rZt2xQYGKgHHnhAixcv1kMPPaRJkyZp3759euqpp5SUlCQvr5q7ePRidQQGBmrChAmaNm2amjVrpvDwcM2YMcOpxvbt2+u2227T+PHj9dxzz6l79+4qKCjQxo0b1aVLF1133XVVrvNvf/ubkpKS9O9//1tt27bVokWLdPTo0Sr3K50dIXrooYe0dOlS+fj4aNKkSerdu7cjEM2cOVM33HCDIiIiNHr0aHl5eWn37t369ttvudoLuAACEGCoOXPmKCwsTMnJyTpw4IAaN26sHj166PHHH9ell16qtWvXatq0aeratauaNm2qCRMm6IknnqjRGitSx7PPPqvjx4/r73//u4KCgjR16lQVFhY69bNixQrNnTtXU6dO1eHDh9WsWTPFxcW5JfxI0t13361du3Zp/PjxatCggRITE90y+iOdXX/06KOPauzYsfr555/Vr18/vfrqq47tw4YN00cffaTZs2drwYIF8vHxUYcOHXTPPfe45fhAfWWzKjKJDgAAUI9wI0QAAGAcAhCAKsvOzlZgYOB5X9nZ2Z4u8byGDx9+3rrnz5/v6fIAVBOmwABU2ZkzZ3Tw4MHzbm/durUaNKidSw4PHz6sU6dOlbutadOmatq0aQ1XBKAmEIAAAIBxmAIDAADGIQABAADjEIAAAIBxCEAAAMA4BCAAAGAcAhAAADAOAQgAABiHAAQAAIzz/wBc1nOoALtgXAAAAABJRU5ErkJggg==p(where='mid')   # beware of syntaxe: F in varianceFraction

###  Plot the first 2 EOFs, PCs and Variance Fraction:  

In [ ]:
# eof = linear_regression_coef(PC,sstanom) = Cov(PC,sstanom)/Var(PC)
# pcscaling=1 (default) --> Var(PC) = 1 --> eof = Cov(PC,sstanom)
eofs = solver.eofsAsCovariance(neofs=2, pcscaling=1)      # beware of syntaxe: A C in eofsAsCovariance
pcs  = solver.pcs(npcs=2, pcscaling=1)
varfrac = solver.varianceFraction(neigs=2)                # beware of syntaxe: F in varianceFraction

# create a 4 pannels figure
fig, axes = plt.subplots(2,2,figsize=(10, 5),constrained_layout=True)
# plot EOF1 and PC1
eofs.sel(mode=0).plot(ax=axes[0,0], cbar_kwargs={'label': '°C for 1 std of PC'})
axes[0,0].set_title('EOF 1: '+str(int(varfrac.values[0]*100))+'%') 
pcs.sel(mode=0).plot(ax=axes[1,0])
axes[1,0].set_title('PC 1')
# plot EOF2 and PC2
eofs.sel(mode=1).plot(ax=axes[0,1], cbar_kwargs={'label': '°C for 1 std of PC'})
axes[0,1].set_title('EOF 2: '+str(int(varfrac.values[1]*100))+'%')
pcs.sel(mode=1).plot(ax=axes[1,1])
axes[1,1].set_title('PC 2')

### Compare EOF and regression on PC

In [ ]:
# linear regression of sstanom against PC1
sstanom.coords["time"] = (("time"), pcs.sel(mode=0).data)  # redefine 'time' coordinates with PC1 time series
linfit = sstanom.polyfit('time', 1)                        # compute the regression with this new "time" 
sstanom.coords["time"] = (("time"), data.time.data)        # put back original time
sstreg = linfit.polyfit_coefficients.isel(degree=0)

# create a 2 pannels figure
fig, axes = plt.subplots(1, 2,figsize=(14, 4),constrained_layout=True)
# plot regression coef
sstreg.where(mask == 1.).plot(ax=axes[0], vmin=-1, vmax=1, cmap='seismic')
axes[0].set_title('Regression coefficient with PC1')
# plot EOF1
eofs.sel(mode=0).plot(ax=axes[1], vmin=-1, vmax=1, cmap='seismic', cbar_kwargs={'label': '°C for 1 std of PC'})
axes[1].set_title('EOF 1: '+str(int(varfrac.values[0]*100))+'%') 

In [ ]:
print(solver.pcs(npcs=10, pcscaling=1).var(dim='time'))


### Composite of extrem positive events, based on PC1:

In [ ]:
pc1 = pcs.sel(mode=0)
pc1.where(pc1 > 2).dropna(dim="time").time

In [ ]:
big = sstanom.where(pc1 > 2)
# create a 2 pannels figure
fig, axes = plt.subplots(1, 2,figsize=(14, 4),constrained_layout=True)
# composite for 1982-1983
big82 = big.sel(time=slice("1982-12","1983-02")).mean("time")
big82.where(mask == 1.).plot(ax=axes[0], vmin=-4, vmax=4, cmap="RdBu_r")
axes[0].set_title('SST anom, Dec 1982 - Feb 1983 composite')
# composite for 1982-1983
big97 = big.sel(time=slice("1997-12","1998-02")).mean("time")
big97.where(mask == 1.).plot(ax=axes[1], vmin=-4, vmax=4, cmap="RdBu_r")
axes[1].set_title('SST anom, Dec 1997 - Feb 1998 composite')

### Composite of extrem negative events, based on PC1:

In [ ]:
sstanom.where(pc1 < -2).mean("time").where(mask == 1.).plot(vmin=-4, vmax=4, cmap="RdBu_r")

###  Repeat the EOF analysys with SST (not SST anomalies)

In [ ]:
solver = eof(data.sst, weights=weights)   # use data.sst insetad of sstanom

In [ ]:
solver.varianceFraction(neigs=10).plot.step(where='mid')

In [ ]:
eofs = solver.eofsAsCovariance(neofs=2, pcscaling=1)      # beware of syntaxe: A C in eofsAsCovariance
pcs  = solver.pcs(npcs=2, pcscaling=1)
varfrac = solver.varianceFraction(neigs=2)                # beware of syntaxe: F in varianceFraction

# create a 4 pannels figure
fig, axes = plt.subplots(2,2,figsize=(10, 5),constrained_layout=True)
# plot EOF1 and PC1
eofs.sel(mode=0).plot(ax=axes[0,0], cbar_kwargs={'label': '°C for 1 std of PC'}, vmin=-4, vmax=4, cmap="RdBu_r")
axes[0,0].set_title('EOF 1: '+str(int(varfrac.values[0]*100))+'%') 
pcs.sel(mode=0).plot(ax=axes[1,0])
axes[1,0].set_title('PC 1')
# plot EOF2 and PC2
eofs.sel(mode=1).plot(ax=axes[0,1], cbar_kwargs={'label': '°C for 1 std of PC'}, vmin=-4, vmax=4, cmap="RdBu_r")
axes[0,1].set_title('EOF 2: '+str(int(varfrac.values[1]*100))+'%')
pcs.sel(mode=1).plot(ax=axes[1,1])
axes[1,1].set_title('PC 2')